# Classification of emails into spam and ham
In this project we have a dataset containing different emails and we have to classify them into spam and ham

## Importing libraries

In [1]:
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# nltk.download('punkt')
# nltk.download('wordnet')


import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron,SGDClassifier,PassiveAggressiveClassifier
from sklearn.metrics import precision_score,recall_score, f1_score, accuracy_score

#pre-settings 
np.random.seed(42)

## Importing datasets

In [87]:
imported_datasets =pd.read_csv('ham-spam/spamhamdata.csv',sep='\t',header=[0])

In [88]:
imported_datasets.head()

,Type,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Data preprocessing

In [89]:
imported_datasets.describe()

,Type,Email
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [90]:
imported_datasets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Type    5572 non-null   object
 1   Email   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [91]:

punctuations=string.punctuation
stop_words=stopwords.words('english')
lemmatizer=WordNetLemmatizer()

In [92]:
punctuations=string.punctuation
stop_words=stopwords.words('english')
lemmatizer=WordNetLemmatizer()
#convert spam and ham to 0/1
#remove stop words
    # tokenize the text
    # convert text to lowercase
    # remove punctuation
    # remove numerical values
    # stem or lemmatize the words
    # return the cleaned text
def preprocess(obj):
    words = word_tokenize(obj)
    corpus=[word.lower() for word in words if word not in stop_words and word not in punctuations]
    corpus=[lemmatizer.lemmatize(words) for words in corpus]
    corpus=' '.join(corpus)
    return corpus

In [93]:
imported_datasets['Email']=imported_datasets['Email'].apply(preprocess)

In [94]:
imported_datasets['IsSpam']=imported_datasets['Type'].apply(lambda x:1 if x=='spam' else 0)

In [95]:
imported_datasets=imported_datasets.drop('Type',axis=1)

In [96]:
imported_datasets.head(3)

,Email,IsSpam
0,go jurong point crazy .. available bugis n gre...,0
1,ok lar ... joking wif u oni ...,0
2,free entry 2 wkly comp win fa cup final tkts 2...,1


## Data Transformation

### Vectorzation of data

In [97]:
count_vectorizer=CountVectorizer()
tfidf=TfidfVectorizer()
Emails=imported_datasets['Email']
Emails_tfidf=tfidf.fit_transform(Emails)
Emails_count_vectorizer=count_vectorizer.fit_transform(Emails)
print(tfidf.get_feature_names_out())
print(count_vectorizer.get_feature_names_out())

['00' '000' '000pes' ... 'èn' 'ú1' '〨ud']
['00' '000' '000pes' ... 'èn' 'ú1' '〨ud']


In [98]:
Emails_tfidf[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

## Splitting of data and Model Training

In [99]:
x=Emails_tfidf
y=imported_datasets['IsSpam']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2)

In [100]:
# models
# from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,RandomForestClassifier,GradientBoostingClassifier
# from sklearn.naive_bayes import MultinomialNB,BernoulliNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.linear_model import Perceptron,SGDClassifier,PassiveAggressiveClassifier

#ensemble
bagging=BaggingClassifier()
ada=AdaBoostClassifier()
extra_trees=ExtraTreesClassifier()
gradient=GradientBoostingClassifier()
random_forest=RandomForestClassifier()

#naive bayes
multinomial=MultinomialNB()
bernoulli=BernoulliNB()

#neural network
mlp=MLPClassifier()

#linear model
perceptron=Perceptron()
sgd=SGDClassifier()
passive_aggressive=PassiveAggressiveClassifier()

In [101]:
clfs={
    'Bagging':bagging,
    'AdaBoost':ada,
    'ExtraTrees':extra_trees,
    'GradientBoosting':gradient,
    'RandomForest':random_forest,
    'MultinomialNB':multinomial,
    'BernoulliNB':bernoulli,
    'MLP':mlp,
    'Perceptron':perceptron,
    'SGD':sgd,
    'PassiveAggressiveClassifier':passive_aggressive
}

In [105]:
import sklearn.metrics

for name,clf in clfs.items():
    clf.fit(xtrain,ytrain)
    print(f'{name} trained')
    ypred=clf.predict(xtest)
    print(f'{name} accuracy: {sklearn.metrics.accuracy_score(ytest,ypred)}')
    print(f'{name} precision: {precision_score(ytest,ypred)}')
    print(f'{name} recall: {recall_score(ytest,ypred)}')
    print(f'{name} f1-score: {f1_score(ytest,ypred)}')
    print(f'{name} confusion matrix: ')
    print(sklearn.metrics.confusion_matrix(ytest,ypred))

Bagging trained
Bagging accuracy: 0.9739910313901345
Bagging precision: 0.96875
Bagging recall: 0.8322147651006712
Bagging f1-score: 0.8953068592057761
Bagging confusion matrix: 
[[962   4]
 [ 25 124]]


d:\IDEs\anaconda\envs\email-ham-spam\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost trained
AdaBoost accuracy: 0.9811659192825112
AdaBoost precision: 0.9507042253521126
AdaBoost recall: 0.9060402684563759
AdaBoost f1-score: 0.9278350515463918
AdaBoost confusion matrix: 
[[959   7]
 [ 14 135]]
ExtraTrees trained
ExtraTrees accuracy: 0.9802690582959641
ExtraTrees precision: 1.0
ExtraTrees recall: 0.8523489932885906
ExtraTrees f1-score: 0.9202898550724637
ExtraTrees confusion matrix: 
[[966   0]
 [ 22 127]]
GradientBoosting trained
GradientBoosting accuracy: 0.9766816143497757
GradientBoosting precision: 0.984251968503937
GradientBoosting recall: 0.8389261744966443
GradientBoosting f1-score: 0.9057971014492754
GradientBoosting confusion matrix: 
[[964   2]
 [ 24 125]]
RandomForest trained
RandomForest accuracy: 0.9838565022421525
RandomForest precision: 1.0
RandomForest recall: 0.8791946308724832
RandomForest f1-score: 0.9357142857142857
RandomForest confusion matrix: 
[[966   0]
 [ 18 131]]
MultinomialNB trained
MultinomialNB accuracy: 0.9739910313901345
Multin

## Tuning the models

In [110]:
features=len(count_vectorizer.get_feature_names_out())

In [112]:
nEstimators=int(features*0.5)
print(nEstimators)

4093


In [113]:
#ensemble
bagging=BaggingClassifier(n_estimators=nEstimators)
ada=AdaBoostClassifier(n_estimators=nEstimators)
extra_trees=ExtraTreesClassifier(n_estimators=nEstimators)
gradient=GradientBoostingClassifier(n_estimators=nEstimators)
random_forest=RandomForestClassifier(n_estimators=nEstimators)

#naive bayes
multinomial=MultinomialNB()
bernoulli=BernoulliNB()

#neural network
mlp=MLPClassifier(learning_rate='adaptive',verbose=True)

#linear model
perceptron=Perceptron()
sgd=SGDClassifier()
passive_aggressive=PassiveAggressiveClassifier()

In [114]:
for name,clf in clfs.items():
    clf.fit(xtrain,ytrain)
    print(f'{name} trained')
    ypred=clf.predict(xtest)
    print(f'{name} accuracy: {sklearn.metrics.accuracy_score(ytest,ypred)}')
    print(f'{name} precision: {precision_score(ytest,ypred)}')
    print(f'{name} recall: {recall_score(ytest,ypred)}')
    print(f'{name} f1-score: {f1_score(ytest,ypred)}')
    print(f'{name} confusion matrix: ')
    print(sklearn.metrics.confusion_matrix(ytest,ypred))

Bagging trained
Bagging accuracy: 0.9766816143497757
Bagging precision: 0.9555555555555556
Bagging recall: 0.8657718120805369
Bagging f1-score: 0.9084507042253521
Bagging confusion matrix: 
[[960   6]
 [ 20 129]]


d:\IDEs\anaconda\envs\email-ham-spam\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost trained
AdaBoost accuracy: 0.9811659192825112
AdaBoost precision: 0.9507042253521126
AdaBoost recall: 0.9060402684563759
AdaBoost f1-score: 0.9278350515463918
AdaBoost confusion matrix: 
[[959   7]
 [ 14 135]]
ExtraTrees trained
ExtraTrees accuracy: 0.9820627802690582
ExtraTrees precision: 1.0
ExtraTrees recall: 0.8657718120805369
ExtraTrees f1-score: 0.9280575539568345
ExtraTrees confusion matrix: 
[[966   0]
 [ 20 129]]
GradientBoosting trained
GradientBoosting accuracy: 0.9766816143497757
GradientBoosting precision: 0.984251968503937
GradientBoosting recall: 0.8389261744966443
GradientBoosting f1-score: 0.9057971014492754
GradientBoosting confusion matrix: 
[[964   2]
 [ 24 125]]
RandomForest trained
RandomForest accuracy: 0.9820627802690582
RandomForest precision: 1.0
RandomForest recall: 0.8657718120805369
RandomForest f1-score: 0.9280575539568345
RandomForest confusion matrix: 
[[966   0]
 [ 20 129]]
MultinomialNB trained
MultinomialNB accuracy: 0.9739910313901345
Multin

## Exporting model
We can see that SGDClassifier is best

In [118]:
saved_model=pickle.dump(sgd,open('model.pkl','wb'))